0. 왼쪽의 폴더 모양 아이콘 클릭하세요. 마우스 왼쪽 클릭해서 '새 폴더'클릭 - 폴더 이름은 'pdf'로 변경해주세요. 그 폴더에 pdf 파일들 전부 끌어 넣으세요.


1. 프로그램 설치

In [1]:
!apt-get -y install -qq tesseract-ocr tesseract-ocr-kor
!pip -q install pymupdf pillow pytesseract google-api-python-client google-auth-httplib2 google-auth-oauthlib opencv-python-headless

Selecting previously unselected package tesseract-ocr-kor.
(Reading database ... 126371 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-kor_1%3a4.00~git30-7274cfa-1.1_all.deb ...
Unpacking tesseract-ocr-kor (1:4.00~git30-7274cfa-1.1) ...
Setting up tesseract-ocr-kor (1:4.00~git30-7274cfa-1.1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 68.8 MB/s eta 0:00:00


2. PDF -> JPG

In [2]:
import os, re
import numpy as np
import cv2
import fitz
from PIL import Image
import pytesseract
from pytesseract import Output

PDF_DIR = "/content/pdf"
JPG_DIR = "/content/jpg"
os.makedirs(JPG_DIR, exist_ok=True)

ZOOM = 2.6
ROW_KEYWORDS = ["comment", "코멘트"]
OCR_LANGS = "eng+kor"
OCR_MIN_CONF = 50
OCR_PSM = 6
MIN_HLINE_RATIO = 0.55
LEFT_BAND_RATIO = 0.4
ROW_PAD_UP_PT = 2
ROW_PAD_DOWN_PT = 2

def _norm_token(s: str) -> str:
    return re.sub(r"[^0-9a-zA-Z가-힣]", "", (s or "").lower())

def _opencv_horizontal_lines(gray: np.ndarray, min_len_ratio: float) -> list[int]:
    thr = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    binimg = 255 - thr if np.mean(thr) > 128 else thr
    W = gray.shape[1]
    k = max(1, W // 60)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (k, 1))
    morph = cv2.morphologyEx(binimg, cv2.MORPH_OPEN, kernel, iterations=1)
    contours, _ = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ys = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if w >= int(W * min_len_ratio) and h <= 4:
            ys.append(y)
    return sorted(set(ys))

def _ocr_has_keyword(pil_img: Image.Image) -> bool:
    data = pytesseract.image_to_data(
        pil_img, lang=OCR_LANGS, config=f"--psm {OCR_PSM}", output_type=Output.DICT
    )
    for i in range(len(data["text"])):
        txt = _norm_token(data["text"][i])
        if not txt:
            continue
        try:
            conf = float(data["conf"][i])
        except:
            conf = 0
        if conf < OCR_MIN_CONF:
            continue
        for kw in ROW_KEYWORDS:
            if _norm_token(kw) in txt:
                return True
    return False

def find_comment_row_band(page: fitz.Page):
    mat = fitz.Matrix(ZOOM, ZOOM)
    pix = page.get_pixmap(matrix=mat, alpha=False)
    Wpx, Hpx = pix.width, pix.height
    img = Image.frombytes("RGB", (Wpx, Hpx), pix.samples)
    gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)

    y_lines = _opencv_horizontal_lines(gray, MIN_HLINE_RATIO)
    if len(y_lines) < 2:
        return None
    y_lines = sorted(y_lines)

    left_w = int(Wpx * LEFT_BAND_RATIO)
    for i in range(len(y_lines)-1):
        top_px, bot_px = y_lines[i], y_lines[i+1]
        if bot_px - top_px < 8:
            continue
        crop = img.crop((0, top_px, left_w, bot_px))
        if _ocr_has_keyword(crop):
            top_pt = max(page.rect.y0, (top_px / ZOOM) - ROW_PAD_UP_PT)
            bot_pt = min(page.rect.y1, (bot_px / ZOOM) + ROW_PAD_DOWN_PT)
            return (top_pt, bot_pt)
    return None

def render_page_cut_band(page: fitz.Page, band, zoom: float = ZOOM):
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat, alpha=False)
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    if band:
        px_per_pt = zoom
        top_px = int(max(0, round(band[0] * px_per_pt)))
        bot_px = int(min(img.height, round(band[1] * px_per_pt)))

        # 위쪽 + 아래쪽
        upper = img.crop((0, 0, img.width, top_px))
        lower = img.crop((0, bot_px, img.width, img.height))

        # 두 부분 합치기
        new_h = upper.height + lower.height
        new_img = Image.new("RGB", (img.width, new_h), (255,255,255))
        new_img.paste(upper, (0, 0))
        new_img.paste(lower, (0, upper.height))
        return new_img

    return img

In [3]:
for pdf_file in os.listdir(PDF_DIR):
    if not pdf_file.lower().endswith(".pdf"):
        continue
    pdf_path = os.path.join(PDF_DIR, pdf_file)
    doc = fitz.open(pdf_path)
    if len(doc) == 0:
        continue
    page = doc[0]

    band = find_comment_row_band(page)
    final_img = render_page_cut_band(page, band)

    base = os.path.splitext(pdf_file)[0]
    final_img.save(os.path.join(JPG_DIR, f"{base}_학부모.jpg"), "JPEG", quality=95)

    doc.close()

print("✅ 변환 완료!")

✅ 변환 완료!


3. JPG 폴더 저장 - 저장된 폴더는 zip 압축 상태 입니다. 압축을 풀면 jpg 사진을 사용할 수 있습니다.

In [5]:
import shutil
from google.colab import files

# jpg 폴더를 zip으로 압축
shutil.make_archive("/content/jpg_folder", 'zip', JPG_DIR)

# 로컬 다운로드
files.download("/content/jpg_folder.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

4. 텍스트 추출

In [6]:
import os, re
import fitz  # PyMuPDF

# ==== 경로 ====
PDF_DIR = "/content/pdf"
TXT_DIR = "/content/txt"
os.makedirs(TXT_DIR, exist_ok=True)

# ==== 옵션 ====
ROW_KEYWORDS = ["comment", "코멘트"]

# ==== 유틸 ====
def _norm_token(s: str) -> str:
    return re.sub(r"[^0-9a-zA-Z가-힣]", "", (s or "").lower())

def extract_comment_text(page: fitz.Page) -> str:
    """
    텍스트 PDF에서 코멘트 라벨 옆 칸의 전체 텍스트 추출
    """
    blocks = page.get_text("blocks")  # (x0, y0, x1, y1, text, block_no, ...)
    label_block = None

    # 1) 코멘트 라벨 블록 찾기
    for b in blocks:
        x0, y0, x1, y1, txt, *_ = b
        norm = _norm_token(txt)
        if any(_norm_token(kw) in norm for kw in ROW_KEYWORDS):
            label_block = (x0, y0, x1, y1)
            break

    if label_block is None:
        return "코멘트 없음"

    _, y0, x1_label, y1 = label_block

    # 2) 라벨과 같은 세로 영역(y범위)에 있고, 라벨 오른쪽(x0 > x1_label)인 블록 수집
    comment_blocks = []
    for b in blocks:
        x0, by0, x1, by1, txt, *_ = b
        if x0 > x1_label and not _norm_token(txt) in [_norm_token(k) for k in ROW_KEYWORDS]:
            # 같은 행 또는 약간 겹치는 범위로 간주
            if (by0 >= y0 - 5) and (by1 <= page.rect.y1 + 5):
                comment_blocks.append(b)

    # 3) y0, x0 기준으로 정렬
    comment_blocks.sort(key=lambda b: (round(b[1]), b[0]))

    # 4) 텍스트만 추출
    texts = [b[4].strip() for b in comment_blocks if b[4].strip()]
    text = " ".join(texts)
    text = re.sub(r"\s+", " ", text).strip()

    return text if text else "숙제를 해오지 않았습니다."

#### 여기에서 학원이름과 날짜 수정해주셔야 합니다.

In [7]:
# ==== 실행 ====
results = {}

for pdf_file in os.listdir(PDF_DIR):
    if not pdf_file.lower().endswith(".pdf"):
        continue

    base = os.path.splitext(pdf_file)[0]
    m = re.match(r".*_(TEST|HW)_(.+)", base)
    if not m:
        continue
    kind, name = m.groups()

    doc = fitz.open(os.path.join(PDF_DIR, pdf_file))
    if len(doc) == 0:
        continue
    page = doc[0]

    comment_text = extract_comment_text(page)

    if name not in results:
        results[name] = {}
    results[name][kind] = comment_text

    doc.close()

# ==== txt 저장 ====
for name, comments in results.items():
    txt_path = os.path.join(TXT_DIR, f"{name}.txt")
    with open(txt_path, "w", encoding="utf-8") as f:


############################################################# 학원 위치 수정 / 날짜 수정 #####################################################################################
        f.write("안녕하세요. 중계 매시브학원 국어 이동인T입니다.\n\n")
        f.write("2025. 08. 24. TEST 채점 결과 발송드립니다.\n\n")
        f.write("--- TEST 코멘트 ---\n")
        f.write(comments.get("TEST", "코멘트 없음") + "\n\n")
        f.write("--- HW 코멘트 ---\n")
        f.write(comments.get("HW", "숙제를 해오지 않았습니다") + "\n")

print("✅ 모든 학생 코멘트가 txt로 저장되었습니다.")

✅ 모든 학생 코멘트가 txt로 저장되었습니다.


5. txt 폴더 저장 - 마찬가지로 zip 형태로 저장됩니다. 압축을 풀면 이름으로 저장된 txt 파일을 확인할 수 있습니다. 곧바로 복사해서 알리고로 쏘면 됩니다.

In [8]:
import shutil
from google.colab import files

# txt 폴더를 zip으로 압축
shutil.make_archive("/content/txt_folder", 'zip', TXT_DIR)

# 로컬 다운로드
files.download("/content/txt_folder.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>